# Team Star Average
Below are graphs that show the current star averages for each team.

In [1]:
from blaseball_mike.models import SimulationData, League
from statistics import mean
import pandas
from IPython.display import display, Markdown
from plotly.graph_objects import Figure
import plotly.io as _pio
_pio.renderers.default = "notebook_connected"

In [2]:
sim = SimulationData.load()
display(Markdown(f"Last updated Short Circuit 1, Season {sim.season}, Day {sim.day}"))

Last updated Short Circuit 1, Season 1, Day 0

In [3]:
def parse_emoji(val):
    try:
        return chr(int(val, 16))
    except ValueError:
        return val

# Get a list of all the teams we care about
teams = list(League.load().teams.values())
teams.sort(key=lambda x: x.nickname)

table = pandas.DataFrame()
for team in teams:
    bat_avg = mean([x.get_hitting_stars() for x in team.lineup])
    pitch_avg = mean([x.get_pitching_stars() for x in team.rotation])
    base_avg = mean([x.get_baserunning_stars() for x in team.lineup])
    def_avg = mean([x.get_defense_stars() for x in team.lineup])

    table = table.append({"team": team.nickname, "emoji": parse_emoji(team.emoji), "color": team.main_color,
                          "batting": bat_avg, "pitching": pitch_avg, "baserunning": base_avg, "defense": def_avg},
                          ignore_index=True)

In [4]:

# Plot it
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

# Make the graph layout & fancy play/pause buttons
fig_dict["layout"]["title"] = "Average Stars - Batting vs Pitching"
fig_dict["layout"]["xaxis"] = {"range": [1, 3], "title": "Batting Stars"}
fig_dict["layout"]["yaxis"] = {"range": [0.5, 3], "title": "Pitching Stars"}
fig_dict["layout"]["height"] = 675
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 250, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 250, "easing": "quadratic-in-out"}}],
                "label": "▶",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate", "transition": {"duration": 0}}],
                "label": "⏸",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 67},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 14},
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 250, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

frame = {"data": [], "name": f"Data"}
for team in table["team"].unique():
    teamset = table[table["team"] == team]
    data_dict = {
        "x": list(teamset["batting"]),
        "y": list(teamset["pitching"]),
        "mode": "markers+text",
        "marker": {
            "color": teamset["color"],
            "size": 36,
            "opacity": 1,
        },
        "name": team,
        "text": teamset["emoji"],
        "textposition": "middle center",
        "textfont_size": 20
    }

    fig_dict["data"].append(data_dict)
    frame["data"].append(data_dict)

fig_dict["frames"].append(frame)

slider_step = {
    "args": [
        [frame["name"]],
        {
            "frame": {"duration": 250, "redraw": False},
            "mode": "immediate",
            "transition": {"duration": 250}
        }
    ],
    "label": frame["name"],
    "method": "animate"
}
sliders_dict["steps"].append(slider_step)

fig_dict["layout"]["sliders"] = [sliders_dict]

fig = Figure(fig_dict)
fig.show()


In [5]:
# Do Defense & Baserunning too

fig_dict["layout"]["title"] = "Average Stars - Defense vs Baserunning"
fig_dict["layout"]["xaxis"] = {"range": [1.5, 3.5], "title": "Defense Stars"}
fig_dict["layout"]["yaxis"] = {"range": [1.5, 3.5], "title": "Baserunning Stars"}
fig_dict["data"] = []
fig_dict["frames"] = []
sliders_dict["steps"] = []


frame = {"data": [], "name": f"Data"}
for team in table["team"].unique():
    teamset = table[table["team"] == team]
    data_dict = {
        "x": list(teamset["defense"]),
        "y": list(teamset["baserunning"]),
        "mode": "markers+text",
        "marker": {
            "color": teamset["color"],
            "size": 36,
            "opacity": 1,
        },
        "name": team,
        "text": teamset["emoji"],
        "textposition": "middle center",
        "textfont_size": 20
    }

    fig_dict["data"].append(data_dict)
    frame["data"].append(data_dict)

fig_dict["frames"].append(frame)

slider_step = {
    "args": [
        [frame["name"]],
        {
            "frame": {"duration": 250, "redraw": False},
            "mode": "immediate",
            "transition": {"duration": 250}
        }
    ],
    "label": frame["name"],
    "method": "animate"
}
sliders_dict["steps"].append(slider_step)

fig_dict["layout"]["sliders"] = [sliders_dict]

fig = Figure(fig_dict)
fig.show()